In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.inspection import permutation_importance

from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from ctgan import CTGAN

### Read data

In [11]:
#oversampling data
original_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\original_data.csv")
original_data = original_data.drop(columns=["Unnamed: 0"])
smote1_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\smote_data.csv")
GAN1_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\GAN_data.csv")
borderline1_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\borderline_data.csv")
smote2_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\smote3_data.csv")
GAN2_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\GAN3_data.csv")    
borderline2_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\borderline3_data.csv")

# synthetic data only generated by GAN, SMOTE, BorderlineSMOTE  ->  to analysis
smote_data = pd.concat([smote1_data, smote2_data, original_data], axis=0)
GAN_data = pd.concat([GAN1_data, GAN2_data, original_data], axis=0)
borderline_data = pd.concat([borderline1_data, borderline2_data, original_data], axis=0)

#synthetic data generated in 1/3 by GAN, SMOTE, BorderlineSMOTE and sum with original data -> to analysis
mix_data = pd.concat([smote1_data, GAN1_data, borderline1_data, original_data], axis=0)

# test data
X_test = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\test\\X_test.csv")
y_test = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\test\\y_test.csv")

#undersampling data for gan, smote, borderline
cc_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\clustercentroids_data.csv")
if_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\isolationforest_data.csv")
nm_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\nearmiss_data.csv")
median_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\median_data.csv")
lof_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\new\\localoutlierfactor_data.csv")




In [8]:
mix_data = pd.concat([smote1_data, GAN1_data, borderline1_data, original_data], axis=0)

### Dictionary


In [20]:
data = {}

data

data["CC_smote_data"] = smote_data
data["CC_GAN_data"] = GAN_data
data["CC_borderline_data"] = borderline_data
data["CC_mix_data"] = mix_data
data["IF_smote_data"] = smote_data
data["IF_GAN_data"] = GAN_data
data["IF_borderline_data"] = borderline_data
data["IF_mix_data"] = mix_data
data["NM_smote_data"] = smote_data
data["NM_GAN_data"] = GAN_data
data["NM_borderline_data"] = borderline_data
data["NM_mix_data"] = mix_data
data["Median_smote_data"] = smote_data
data["Median_GAN_data"] = GAN_data
data["Median_borderline_data"] = borderline_data
data["Median_mix_data"] = mix_data
data["LOF_smote_data"] = smote_data
data["LOF_GAN_data"] = GAN_data
data["LOF_borderline_data"] = borderline_data
data["LOF_mix_data"] = mix_data

### Distribiution

In [21]:
output_xlsx = "D:\\ml\\undersampling_data\\reports\\ssh\\distribiution\\distribiution.xlsx"
output_txt = "D:\\ml\\undersampling_data\\reports\\ssh\\distribiution\\distribiution.txt"
os.makedirs(output_xlsx, exist_ok=True)
os.makedirs(output_txt, exist_ok=True)

for name, data in data.items():
    
    data = pd.DataFrame(data)
    results = data["source"].value_counts(normalize=True) * 100
    print(f"Data: {name}")
    print(f"Distribiution:\n{results}")
    
    results.to_csv(output_txt, sep="\t", index=False)
    results.to_excel(output_xlsx, index=False, sheet_name="Arkusz1")
    
    
#compare collect and sum_data_NM and copy column source to collect, if rows are the same
""" collect = collect.merge(sum_data_NM, on=collect.columns.tolist(), how='left', indicator=True)
collect = collect.drop(columns=["_merge"])
collect = collect.drop_duplicates()
print(collect)
print(collect["source"].value_counts()) """

Data: CC_smote_data
Distribiution:
source
smote       61.523438
original    38.476562
Name: proportion, dtype: float64


PermissionError: [Errno 13] Permission denied: 'D:\\ml\\undersampling_data\\reports\\ssh\\distribiution\\distribiution.txt'